In [2]:
import sys
sys.path.append('../src')

In [3]:
sys.path

['/home/boris/Projects/Voice_Assistant_for_Voice_Anomaly_Persons/Multi-lingual Phoneme Recognition/notebooks',
 '/home/boris/miniconda3/envs/voice-speaker/lib/python310.zip',
 '/home/boris/miniconda3/envs/voice-speaker/lib/python3.10',
 '/home/boris/miniconda3/envs/voice-speaker/lib/python3.10/lib-dynload',
 '',
 '/home/boris/.local/lib/python3.10/site-packages',
 '/home/boris/miniconda3/envs/voice-speaker/lib/python3.10/site-packages',
 '/tmp/tmp57_ujoaa',
 '../src']

In [4]:
import logging
logging.basicConfig(filename='val.log',
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger()

In [5]:
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
import tqdm
import pytorch_lightning as pl
from torchinfo import summary

In [6]:
import json

In [7]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [8]:
f = open("/home/boris/Projects/Voice_Assistant_for_Voice_Anomaly_Persons/Multi-lingual Phoneme Recognition/models/processor/vocab.json")
phonemes: dict[str, int] = json.load(f)
f.close()

phonemes_reverse ={j:i for i, j in phonemes.items()}

In [9]:
params = {
    'batch_size': 4,
    'phonemes': 392
}
logger.info(str(params))

In [142]:
df = pd.read_csv('/home/boris/Projects/Voice_Assistant_for_Voice_Anomaly_Persons/Multi-lingual Phoneme Recognition/data/датасет3.csv').values.tolist()
df = pd.read_csv('/home/boris/Projects/Voice_Assistant_for_Voice_Anomaly_Persons/Multi-lingual Phoneme Recognition/data/xls-r_dataset.csv').values.tolist()

train_df, test_df = train_test_split(df, test_size=0.1)
train_df, val_df = train_test_split(train_df)

In [143]:
train_loader = DataLoader(train_df, params["batch_size"], True)
val_loader = DataLoader(val_df, params["batch_size"], True)
test_loader = DataLoader(test_df, params["batch_size"], True)

Let see what is the type of batches

In [144]:
batch = next(iter(train_loader))
batch

[tensor([192, 632,  73,  41]),
 ('Варить кофе с солью',
  'Смерть это необратимый финал нашей жизни',
  'Матёрый',
  'отмечать'),
 ('v a ɡ r i t k o f e s t o n ʒ o',
  's y n e k r i t ɛ t e n e a v u k k r a t i l m i f i n ɑ n e f e ʒ e z ə n i',
  'm aː t o k k r i x',
  'a t ʌ m i ʃ ɛ l tʃ')]

so, we've `ids: list[int], texts: list['str'], phonemes: list[str]`

In [92]:
abc = "?абвгдеёжзийклмнопрстуфхшщчцьыъэюя"
def vectorize_str(labels: tuple[str]):
    lengths = torch.LongTensor(size=(len(labels),))
    letters = torch.zeros(size=(len(labels),  max(map(len, labels)), len(abc)), dtype=torch.double)
    
    for i, label in enumerate(labels):
        lengths[i] = len(label)
        j=0
        for c in label.lower():
            if not c in abc:
                lengths[i]-=1
            else:
                letters[i,j, abc.index(c)]=1
                # letters[i,j]= abc.index(c)
                j+=1
    
    return letters, lengths
def decode_str(X, pred=True):
    r = []
    for i in range(X.shape[0]):
        s = []
        s1 = []
        for j in range(X.shape[1]):
            if (ix:=X[i,j].argmax()) >= 1:
                s.append(abc[ix])
            s1.append(abc[X[i,j,1:].argmax()+1])
        if True:
            r.append(''.join(s)+'/'+''.join(s1))
        else:
            r.append(''.join(s))
    return r

In [93]:
k = vectorize_str(batch[1])[0]
decode_str(k)

['сколькоможнотакделать/сколькоможнотакделатьаааааааааааааааааааааа',
 'лазурноеморе/лазурноемореааааааааааааааааааааааааааааааа',
 'зубнаяпаста/зубнаяпастааааааааааааааааааааааааааааааааа',
 'нонапоездепочтиникогданебываетаварий/нонапоездепочтиникогданебываетаварийааааааа']

In [94]:
def vectorize_phonemes(labels: tuple[str]):
    lengths = torch.LongTensor(size=(len(labels),))
    letters = torch.zeros(
        size=(
            len(labels),
            max(map(lambda x: len(x.split()), labels)),
            params['phonemes']
        ),
        dtype=torch.float
    )
    # letters = torch.zeros(size=(len(labels),  max(map(len, labels))), dtype=float)

    for i, label in enumerate(labels):
        lengths[i] = len(label.split())
        for j, c in enumerate(label.split()):
            letters[i,j, phonemes[c]]=1
    
    return letters, lengths
def decode_phonemes(X):
    r = []
    for i in range(X.shape[0]):
        s = []
        for j in range(X.shape[1]):
            if X[i,j].argmax() >= 0:
                s.append(phonemes_reverse[int(X[i,j].argmax())])
        r.append(''.join(s))
    return r

In [95]:
batch[2]

('s k r o l k a m o ʒ ɯ n a t a k n t e l ɯ t',
 'l a z o k r o n n a ɡ e m o k r i',
 'z o v o n aː j a p a s t a',
 'n o n a p o e z i d e p e tʃ t i n i k a ɡ d a n e v o v a e t a v a k r i x')

In [96]:
decode_phonemes(vectorize_phonemes(batch[2])[0])

['skrolkamoʒɯnatakntelɯt<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 'lazokronnaɡemokri<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 'zovonaːjapasta<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 'nonapoezidepetʃtinikaɡdanevovaetavakrix']

Test of preprocess correctness

In [97]:
class Reshape(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        self.shape = args
    def forward(self, x):
        return x.reshape(*self.shape)

In [151]:
class Conv1DCorrector(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()
        self.conv = nn.Conv1d(
            params['phonemes'],
            20,
            3,
            padding=1
        )

        self.lstm = nn.LSTM(20, 40, batch_first=True)
        self.linear = nn.Linear(40, len(abc))

    def forward(self, x: torch.Tensor):
        # x: batch x len x phonemes
        x = x.permute(0,2,1)
        # x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])# x: batch x 1 x phonhemes x len
        x = self.conv(x)
        x = x.permute(0,2, 1)
        x, (h, c) = self.lstm(x)
        x = self.linear(x)
        return F.log_softmax(x, -1)
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    def training_step(self, train_batch, batch_idx):
        labels, lengths = vectorize_str(train_batch[1])
        phonemes_vec, _ = vectorize_phonemes(train_batch[2])
        
        prediction = self(phonemes_vec.to(device))
        
        ilengths = lengths.clone()
        for i in range(prediction.shape[0]):
            ilengths[i] = prediction.shape[1]
        
        lv = F.ctc_loss(prediction.permute(1, 0, 2), labels.argmax(dim=-1), ilengths, lengths, zero_infinity=True)
        
        if batch_idx==0:
            logging.info("train \"%s\" recognized as \"%s\"", '|'.join(decode_str(labels, False)), '|'.join(decode_str(prediction)))
        
        self.log('train_loss', lv, True, batch_size=len(train_batch))
        return lv
    def validation_step(self, train_batch, batch_idx):
        labels, lengths = vectorize_str(train_batch[1])
        phonemes_vec, _ = vectorize_phonemes(train_batch[2])
        
        prediction = self(phonemes_vec.to(device))
        
        ilengths = lengths.clone()
        for i in range(prediction.shape[0]):
            ilengths[i] = prediction.shape[1]
        
        lv = F.ctc_loss(prediction.permute(1, 0, 2), labels.argmax(dim=-1), ilengths, lengths, zero_infinity=True)

        if batch_idx==0:
            logging.info("val \"%s\" recognized as \"%s\"", '|'.join(decode_str(labels, False)), '|'.join(decode_str(prediction)))
        
        self.log('val_loss', lv, True, batch_size=len(train_batch))


In [152]:
model = Conv1DCorrector()

In [153]:
db = vectorize_phonemes(next(iter(train_loader))[2])[0]
db.shape

torch.Size([4, 20, 392])

In [154]:
summary(model, input_data=db, verbose=1, col_names=['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Conv1DCorrector                          [4, 20, 392]              [4, 20, 34]               --
├─Conv1d: 1-1                            [4, 392, 20]              [4, 20, 20]               23,540
├─LSTM: 1-2                              [4, 20, 20]               [4, 20, 40]               9,920
├─Linear: 1-3                            [4, 20, 40]               [4, 20, 34]               1,394
Total params: 34,854
Trainable params: 34,854
Non-trainable params: 0
Total mult-adds (M): 2.68
Input size (MB): 0.13
Forward/backward pass size (MB): 0.06
Params size (MB): 0.14
Estimated Total Size (MB): 0.33


/home/boris/miniconda3/envs/voice-speaker/lib/python3.10/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/boris/miniconda3/envs/voice-speaker/lib/python3.10/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Conv1DCorrector                          [4, 20, 392]              [4, 20, 34]               --
├─Conv1d: 1-1                            [4, 392, 20]              [4, 20, 20]               23,540
├─LSTM: 1-2                              [4, 20, 20]               [4, 20, 40]               9,920
├─Linear: 1-3                            [4, 20, 40]               [4, 20, 34]               1,394
Total params: 34,854
Trainable params: 34,854
Non-trainable params: 0
Total mult-adds (M): 2.68
Input size (MB): 0.13
Forward/backward pass size (MB): 0.06
Params size (MB): 0.14
Estimated Total Size (MB): 0.33

In [155]:
b = next(iter(train_loader))

In [156]:
b

[tensor([237, 494, 176, 217]),
 ('Умничка', 'Производство', 'Дайте волку банан', 'Маленькие женщины'),
 ('o m n i tʃ k a',
  'd e j t e r ə n ɛ θ t',
  'd aɪ t e v o l k o v a n a n',
  'm a l ɪ n v k eː j ʒ n ʃ e n ɪ')]

In [157]:
model.validation_step(b, 0)

/home/boris/miniconda3/envs/voice-speaker/lib/python3.10/site-packages/pytorch_lightning/core/module.py:410: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


In [158]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")

trainer = pl.Trainer(max_epochs=400, logger=logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [159]:
trainer.fit(model, train_loader, test_loader)


  | Name   | Type   | Params
----------------------------------
0 | conv   | Conv1d | 23.5 K
1 | lstm   | LSTM   | 9.9 K 
2 | linear | Linear | 1.4 K 
----------------------------------
34.9 K    Trainable params
0         Non-trainable params
34.9 K    Total params
0.139     Total estimated model params size (MB)


Epoch 64: 100%|██████████| 129/129 [00:03<00:00, 37.51it/s, v_num=13, train_loss=1.570, val_loss=3.230]

https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html